In [1]:
import sys
import numpy as np
from scipy.sparse import csr_matrix as csr_mat
import time
from multiprocessing import Pool    
from quspin.tools.lanczos import lanczos_full, expm_lanczos
import numba as nb

In [2]:
# auxiliary functions to create basis
def first_state(L,up=False):
        if up:
            n_upspins = L//2+1
        else:
            n_upspins = L//2
        return (1 << n_upspins) - 1
def next_state(state):
        t = (state | (state - 1)) + 1
        return t | ((((t & -t) // (state & -state)) >> 1) - 1)
def last_state(L,up=False):
        if up:
            n_upspins = L//2+1
        else:
            n_upspins = L//2
        return ((1 << n_upspins) - 1) << (L - n_upspins)

#get number of spin states Ns if hole is present  

def N_spinStates(L,hole_present=True):
    if L%2==1:
        raise ValueError('Only even number of sites possible')
    if hole_present:
        N=np.math.factorial(L)/(2*(np.math.factorial(L/2))**2)
    else:
        N=np.math.factorial(L)/((np.math.factorial(L/2))**2)
    return int(N)

# create spin basis for Sz=0
def Sz0basis(L,up=False):
    basis = []
    state = first_state(L,up)
    end_state = last_state(L,up)
    while state <= end_state:
        basis.append(state)
        state = next_state(state)
    return np.array(basis)

# create basis for t-J model
def tJ_basis_down(L):
    basis=np.array([])
    for j in range(0,L):
        basis=np.append(basis,Sz0basis(L-1,up=False)+j*2**(L-1))
    return basis.astype('int32')

#get value of state "state" on site "site"
@nb.jit(nopython=True, cache=True)
def get_site_value(state, site):
    return (state >> site) & 1

@nb.jit(nopython=True, cache=True)
def reducebasis(spin_basis, L, hole_pos):
    """reduces basis by hole up"""
    reduced_basis=[]
    for state in spin_basis:
        if get_site_value(state, hole_pos)==1:
            reduced_basis.append(state)
    return reduced_basis

@nb.jit(nopython=True, cache=True)
def get_hole_pos(state,L):
    return int(state/(2**(L-1)))

def csr_matrix(rows,cols, data, s):
    # takes rows, cols, data as a python list
    return csr_mat((data, (rows, cols)), shape=(s,s))

@nb.jit(nopython=True)
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

@nb.jit(nopython=True, cache=True)
def list_add(list1,a):
    return [x+a for x in list1]

def list_mult(list1,a):
    return [x*a for x in list1]

# create heisenberg chain hamiltonian in csr format
@nb.jit(nopython=True, cache=True)
def make_H_spin_J(L, spin_basis, hole_pos=None, hole_present=True):

    # bonds with periodic bc
    heisenberg_bonds = [(site, (site+1)%L) for site in range(L)]
    h_rows = []
    h_cols = []
    h_data = []

    # run through spin state basis
    for state_index, state in enumerate(spin_basis):
        # diagonal S_z interaction
        diagonal = 0
        if hole_present:
            for bond in heisenberg_bonds:
                if bond[0]!=hole_pos and bond[1]!=hole_pos:
                    if get_site_value(state, bond[0]) == get_site_value(state, bond[1]):
                        diagonal += 1/4
                    else:
                        diagonal -= 1/4
        else:
            for bond in heisenberg_bonds:
                if get_site_value(state, bond[0]) == get_site_value(state, bond[1]):
                    diagonal += 1/4
                else:
                    diagonal -= 1/4
                      
        #constant terms depending if hole is present  
        #Wohlfeld
        if hole_present:
            diagonal -= (L-2)*1/4   #for the constant -J_ii1/4 n_i n_i+1 and E_ii1 ni term
        else:                               #in the Wohlfeld definition E has to be negative
            diagonal -= L*1/4
        h_rows.append(state_index)
        h_cols.append(state_index)
        h_data.append(diagonal)

        # offdiagonal S_x and S_y interaction
        if hole_present:
            for bond in heisenberg_bonds:
                if bond[0]!=hole_pos and bond[1]!=hole_pos:
                    flipmask = (1 << bond[0]) | (1 << bond[1])
                    if get_site_value(state, bond[0]) != get_site_value(state, bond[1]):
                        new_state = state ^ flipmask
                        try:
                            new_state_index = spin_basis.index(new_state)
                            h_rows.append(state_index)
                            h_cols.append(new_state_index)
                            h_data.append(1/2)
                        except:
                            pass
        else:
            for bond in heisenberg_bonds:
                flipmask = (1 << bond[0]) | (1 << bond[1])
                if get_site_value(state, bond[0]) != get_site_value(state, bond[1]):
                    new_state = state ^ flipmask
                    new_state_index = spin_basis.index(new_state)
                    h_rows.append(state_index)
                    h_cols.append(new_state_index)
                    h_data.append(1/2)
    return h_rows, h_cols, h_data

@nb.jit(nopython=True, cache=True)
def make_H_spin_E(L, spin_basis, hole_pos=None, hole_present=True):

    # bonds with periodic bc
    heisenberg_bonds = [(site, (site+1)%L) for site in range(L)]
    h_rows = []
    h_cols = []
    h_data = []

    # run through spin state basis
    for state_index, state in enumerate(spin_basis):
        diagonal = 0
               
        #constant terms depending if hole is present 
        #Wohlfeld
        if hole_present:
            diagonal += (L-1)   #for the constant -J_ii1/4 n_i n_i+1 and E_ii1 ni term
        else:                               #in the Wohlfeld definition E has to be negative
            diagonal += L

        h_rows.append(state_index)
        h_cols.append(state_index)
        h_data.append(diagonal)

    return h_rows, h_cols, h_data


@nb.jit(nopython=True, cache=True)
def make_H_tJ_t(L, basis, Ns_h, spin_basis):
    # the Hamiltonion is build as
    # H = sum_i { t_ii1 [p+_i p_i1 + h.c. ]+ (J_ii1 [S_i S_i1 -1/4 n_i n_i+1]) + E_ii1 n_i }
      
    heisenberg_bonds = [(site, (site+1)%L) for site in range(L)]
    h_rows = []
    h_cols = []
    h_data = []
    
    # add hoppings of hole           
    for j in range(0,(L-1)*Ns_h):
        h_rows.append(j)
        h_cols.append(j+Ns_h)
        h_data.append(1)
        h_rows.append(j+Ns_h)
        h_cols.append(j)
        h_data.append(1)

    spin_part=list(basis[0:Ns_h])             #the spin states of L-1, Sz=+1/2
    for state_id, state in enumerate(spin_part):
        site_0_value = get_site_value(state,0)
        new_state= (state >> 1)+site_0_value*2**(L-2)
        new_state_id = spin_part.index(new_state)
        h_rows.append(state_id+Ns_h*(L-1))
        h_cols.append(new_state_id)
        h_data.append(1)
        h_rows.append(new_state_id)
        h_cols.append(state_id+Ns_h*(L-1))
        h_data.append(1)
           
    return h_rows, h_cols, h_data

@nb.jit(nopython=True, cache=True)
def make_H_tJ_J(L, basis, Ns_h, spin_basis):
    
    heisenberg_bonds = [(site, (site+1)%L) for site in range(L)]

    # add blockdiagonal Heisenberg chain entrys
    for hole_pos in range(0,L):
        reduced_basis=reducebasis(spin_basis, L, hole_pos)
        h_r, h_c, h_d = make_H_spin_J(L, reduced_basis, hole_pos=hole_pos, hole_present=True)
        
        if hole_pos==0:
            h_rows=h_r
            h_cols=h_c
            h_data=h_d
        else:
            h_rows.extend(list_add(h_r,hole_pos*Ns_h))
            h_cols.extend(list_add(h_c,hole_pos*Ns_h))
            h_data.extend(h_d)
              
    return h_rows, h_cols, h_data

@nb.jit(nopython=True, cache=True)
def make_H_tJ_E(L, basis, Ns_h, spin_basis):
    
    heisenberg_bonds = [(site, (site+1)%L) for site in range(L)]

    # add blockdiagonal Heisenberg chain entrys
    for hole_pos in range(0,L):
        reduced_basis=reducebasis(spin_basis, L, hole_pos)
        h_r, h_c, h_d = make_H_spin_E(L, reduced_basis, hole_pos=hole_pos, hole_present=True)
        
        if hole_pos==0:
            h_rows=h_r
            h_cols=h_c
            h_data=h_d
        else:
            h_rows.extend(list_add(h_r,hole_pos*Ns_h))
            h_cols.extend(list_add(h_c,hole_pos*Ns_h))
            h_data.extend(h_d)
            
    return h_rows, h_cols, h_data
        


In [3]:
@nb.jit(nopython=True, cache=True)
def createHole(v1, hole0, k, spin_basis):
    return [
        (v1[state_id] if site == 0 else v1[state_id] * 0)
        if hole0 else
        (v1[state_id] * np.exp(1j * k * site) if site % 2 == 0 else -v1[state_id] * np.exp(1j * k * site))
        for site in range(0, L)
        for state_id, state in enumerate(spin_basis)
        if get_site_value(state, site) == 1
    ]
    
        
def propagater_bash(Nt1, Nt_max, v1, tPhi_ii1, JPhi_ii1 ,EPhi_ii1, spin_basis, basis_tJ, dim_krylov, start_total):
    Ns_nh=N_spinStates(L, hole_present=False)
    Ns_h=N_spinStates(L,hole_present=True)
    v1_nt1=np.zeros((Nt_max,len(v1)),dtype=complex)
    G_t2_k=np.zeros((Nt_max,len(k_all)),dtype=complex)
    run_cal=True
    
    #creating spin Hamiltonian
    h_spin_rows = []
    h_spin_cols = []
    h_spin_r_J, h_spin_c_J, h_spin_d_J = make_H_spin_J(L, typed_spin_basis, hole_present=False)
    h_spin_r_E, h_spin_c_E, h_spin_d_E = make_H_spin_E(L, typed_spin_basis, hole_present=False)
    h_spin_rows.extend(h_spin_r_E)
    h_spin_cols.extend(h_spin_c_E)
    h_spin_rows.extend(h_spin_r_J)
    h_spin_cols.extend(h_spin_c_J)
    
    #creating tJ Hamiltonian
    h_tJ_rows = []
    h_tJ_cols = []
    h_tJ_r_t, h_tJ_c_t, h_tJ_d_t = make_H_tJ_t(L, basis_tJ,  Ns_h, typed_spin_basis)
    h_tJ_r_J, h_tJ_c_J, h_tJ_d_J = make_H_tJ_J(L, basis_tJ,  Ns_h, typed_spin_basis)
    h_tJ_r_E, h_tJ_c_E, h_tJ_d_E = make_H_tJ_E(L, basis_tJ,  Ns_h, typed_spin_basis)
    h_tJ_rows.extend(h_tJ_r_t)
    h_tJ_cols.extend(h_tJ_c_t)
    h_tJ_rows.extend(h_tJ_r_J)
    h_tJ_cols.extend(h_tJ_c_J)
    h_tJ_rows.extend(h_tJ_r_E)
    h_tJ_cols.extend(h_tJ_c_E)
        
    for Ntr in range(Nt1):
        h_spin_data = []
        h_spin_data.extend(list_mult(h_spin_d_E,EPhi_ii1[Ntr]))
        h_spin_data.extend(list_mult(h_spin_d_J,JPhi_ii1[Ntr]))
        H_spin=csr_matrix(h_spin_rows, h_spin_cols, h_spin_data, N_spinStates(L, hole_present=False))
        E_spin ,V_spin ,Q_T_spin = lanczos_full(H_spin,v1,dim_krylov) 
        v1 = expm_lanczos(E_spin,V_spin,Q_T_spin,a=-1j*dt)
        v1_nt1[Ntr,:]=v1
    
    v2=v1
    for Ntr in range(Nt1,Nt_max):
        h_spin_data = []
        h_spin_data.extend(list_mult(h_spin_d_E,EPhi_ii1[Ntr]))
        h_spin_data.extend(list_mult(h_spin_d_J,JPhi_ii1[Ntr]))
        H_spin=csr_matrix(h_spin_rows, h_spin_cols, h_spin_data, N_spinStates(L, hole_present=False))
        E_spin ,V_spin ,Q_T_spin = lanczos_full(H_spin,v2,dim_krylov) 
        v2 = expm_lanczos(E_spin,V_spin,Q_T_spin,a=-1j*dt)
        v1_nt1[Ntr,:]=v2
        
    #create hole at j=0, change basis
    v_tJ=np.array(createHole(v1,True,0,spin_basis))

    #for Nt2>Nt1
    for Nt2 in range(Nt1,Nt_max):
        h_tJ_data = []
        h_tJ_data.extend(list_mult(h_tJ_d_t,tPhi_ii1[Nt2]))
        h_tJ_data.extend(list_mult(h_tJ_d_J,JPhi_ii1[Nt2]))
        h_tJ_data.extend(list_mult(h_tJ_d_E,EPhi_ii1[Nt2]))
        H_tJ=csr_matrix(h_tJ_rows, h_tJ_cols, h_tJ_data, L*Ns_h)
        
        E_tJ ,V_tJ ,Q_T = lanczos_full(H_tJ,v_tJ,dim_krylov) 
        v_tJ = expm_lanczos(E_tJ,V_tJ,Q_T,a=-1j*dt)
        
        #create hole at j, change basis
        for k_id, k in enumerate(k_all):
            phi_pj=np.array(createHole(v1_nt1[Nt2,:],False,k,spin_basis))
            G_t2_k[Nt2,k_id]=np.vdot(phi_pj,v_tJ)
            
        filelog=open(filename_log,'a')
        data_stringlog = str(Nt2) + "\n"
        filelog.write(data_stringlog)
        filelog.close()
        
        end = time.time()
        #82800
        #print(end-start_total)
        if (end-start_total) > 82000:
            np.save('G_t2_k_'+str(Nt1)+'.npy',G_t2_k)
            np.save('v1_nt1_'+str(Nt1)+'.npy',v1_nt1)
            np.save('v1_'+str(Nt1)+'.npy',v1)
            np.save('v_tJ_'+str(Nt1)+'.npy',v_tJ)
            np.save('Nt2_'+str(Nt1)+'.npy',Nt2)
            print(Nt2)
            run_cal=False
            break
            
        
    if run_cal:
        #reset states        
        v_tJ=np.array(createHole(v1,True,0,spin_basis))

        #for Nt2<Nt1
        for Nt2 in range(Nt1-1,-1,-1):
            h_tJ_data = []
            h_tJ_data.extend(list_mult(h_tJ_d_t,tPhi_ii1[Nt2]))
            h_tJ_data.extend(list_mult(h_tJ_d_J,JPhi_ii1[Nt2]))
            h_tJ_data.extend(list_mult(h_tJ_d_E,EPhi_ii1[Nt2]))
            H_tJ=csr_matrix(h_tJ_rows, h_tJ_cols, h_tJ_data, L*Ns_h)
        
            E_tJ ,V_tJ ,Q_T = lanczos_full(H_tJ,v_tJ,dim_krylov) 
            v_tJ = expm_lanczos(E_tJ,V_tJ,Q_T,a=1j*dt)

            #create hole at j, change basis
            for k_id, k in enumerate(k_all):
                phi_pj=np.array(createHole(v1_nt1[Nt2,:],False,k,spin_basis))
                G_t2_k[Nt2,k_id]=np.vdot(phi_pj,v_tJ)

            filelog=open(filename_log,'a')
            data_stringlog = str(Nt2)
            filelog.write(data_stringlog)
            filelog.close()
            
            end = time.time()
            if (end-start_total) > 82000:
                np.save('G_t2_k_'+str(Nt1)+'.npy',G_t2_k)
                np.save('v1_nt1_'+str(Nt1)+'.npy',v1_nt1)
                np.save('v_tJ_'+str(Nt1)+'.npy',v_tJ)
                np.save('Nt2_'+str(Nt1)+'.npy',Nt2)
                print(Nt2)
                run_cal=False
                break

    if run_cal:
        file=open('prop_'+str(Nt1)+'_bash.txt','a')       
        for Nt2 in range(Nt_max):
            for k_id in range(len(k_all)):
                G_t2_k_r=np.real(G_t2_k[Nt2,k_id])
                G_t2_k_im=np.imag(G_t2_k[Nt2,k_id])
                file.write(str(G_t2_k_r) + "\t" + str(G_t2_k_im)+ "\t")
            file.write("\n")
        file.close()
    
    return G_t2_k


In [4]:
"""trail parameters"""
L          = 4
Nt_max     = 10 #20*64       #gives the number of timesteps, only equal to tmax if dt=1
dt         = 0.2  
#mu_pump    = Nt_max/2
sig_pump   = 30
dim_krylov = 4
dPhi       = 2*gaussian(dt*np.arange(Nt_max),2*sig_pump, sig_pump) 

k_all      = np.pi/L*np.arange(-L,L+1,2)
Ns         = N_spinStates(L)

Nt1=10
#Nt1 = int(sys.argv[1])
filename_log="LOG_"+str(Nt1)+".txt"
filelog=open(filename_log,'w')
filelog.close()


In [5]:
# model parameters
t_sigma   = 1.5
t_pi      = 0.83
t_sigmao  = 1.8
t_pio     = 1.0
Delta_x   = 3.0
Delta_y   = 2.8
Delta_z   = 2.2
Delta_xo  = 2.5
Delta_yo  = 3.5
U         = 8.8
U_p       = 4.4
Jb_H      = 1.2
Jc_H      = 0.69
Jp_H      = 0.83
V_dp      = 1.0
epsilon_b = 0.5
epsilon_c = 0.5
eps_b     = 2.2   #\bar{\epsilon}_b
eps_c     = 2.0   #\bar{\epsilon}_c

# parameters for Kugel Khomksi like model
e_a      = (Delta_yo-(Delta_yo**2+8*t_sigmao**2)**(1/2))/2
lambda_a = (Delta_yo-e_a)/(2*t_sigmao**2+(Delta_yo-e_a)**2)**(1/2)
e_c      = (Delta_xo-(Delta_xo**2+8*t_pio**2)**(1/2))/2
lambda_c = (Delta_xo-e_c)/(2*t_pio**2+(Delta_xo-e_c)**2)**(1/2)

J1_p    = 2*(lambda_a*t_sigma)**4/(Delta_x+V_dp+dPhi)**2
J1_m    = 2*(lambda_a*t_sigma)**4/(Delta_x+V_dp-dPhi)**2

Jb2_p   = 2*t_pi**4/(Delta_z+V_dp+dPhi)**2
Jb2_m   = 2*t_pi**4/(Delta_z+V_dp-dPhi)**2

Jb12_p  = 2*(t_pi*lambda_a*t_sigma)**2/((Delta_z+V_dp+dPhi)*(Delta_x+V_dp+dPhi))
Jb12_m  = 2*(t_pi*lambda_a*t_sigma)**2/((Delta_z+V_dp-dPhi)*(Delta_x+V_dp-dPhi))

rb1_p   = 1/(U-3*Jb_H+2*dPhi)
rb1_m   = 1/(U-3*Jb_H-2*dPhi)

rb2_p   = 1/(U-Jb_H+2*dPhi)
rb2_m   = 1/(U-Jb_H-2*dPhi)

ra_p    = 1/(U+2*dPhi)
ra_m    = 1/(U-2*dPhi)

Rb1     = 2/(Delta_x+Delta_z+U_p-3*Jp_H)

Rb2     = 2/(Delta_x+Delta_z+U_p-Jp_H)

Ra      = 1/(2*Delta_x+U_p)



#parameters for t-J model
JPhi = J1_p*ra_p+J1_m*ra_m+Ra*(J1_p**(1/2)+J1_m**(1/2))**2

tPhi = -1/8*((3*rb1_p+3*Rb1+rb2_p+Rb2)*Jb12_p+(3*rb1_m+3*Rb1+rb2_m+Rb2)*Jb12_m) 

EPhi = -(eps_b-Rb1*(Jb12_p+Jb12_m)-rb1_p/2*(J1_p+Jb2_p)-rb1_m/2*(J1_m+Jb2_m))


#print('J:    ' + str(np.round(JPhi,6)))
#print('tb:   ' + str(np.round(tPhi,6)))
#print('E:    ' + str(np.round(EPhi,6)))

In [6]:
"""time dependent krylov time propagation"""
start_total = time.time()
start = time.time()

spin_basis=list(Sz0basis(L).astype('int32'))
typed_spin_basis = nb.typed.List()
[typed_spin_basis.append(x) for x in spin_basis]
phi=np.load('phi_'+str(L)+'.npy')



basis_tJ=tJ_basis_down(L)

propagater_bash(Nt1, Nt_max, phi, tPhi, JPhi ,EPhi, typed_spin_basis, basis_tJ, dim_krylov,start_total)


end = time.time()
filelog=open(filename_log,'a')
data_stringlog = str('CPU time of the propagation')+'     '+str(np.round(end - start,3))+'\n'
filelog.write(data_stringlog)
filelog.close()

In [7]:
"""Testing"""
G_t1_t2_k=np.zeros((Nt_max,Nt_max,len(k_all)),dtype=complex)
for Nt1 in range(Nt_max):
    G_t1_t2_k[Nt1,:,:]=propagater_bash(Nt1, Nt_max, phi, tPhi, JPhi ,EPhi, typed_spin_basis, basis_tJ, dim_krylov,start_total)
    print(Nt1)

#np.save('G_t1_t2_k_L4_Nt100_test.npy',G_t1_t2_k)

0
1
2
3
4
5
6
7
8
9
